# Exploratory Data Analysis

In [7]:
import pandas as pd
import numpy as np

# Load the Data
data = pd.read_csv('../data/machine-1-1.txt', header=None)
labels = pd.read_csv('../data/machine-1-1-label.txt', header=None)

# Rename Columns for Clarity
feature_names = [f"feat_{i}" for i in range(data.shape[1])]
data.columns = feature_names
labels.columns = ['label']

# Combine into one DataFrame
full_df = pd.concat([data, labels], axis=1)

# Split into Train and Test Sets
train_size = int(len(full_df) * 0.70)
train_df = full_df.iloc[:train_size].reset_index(drop=True)
test_df  = full_df.iloc[train_size:].reset_index(drop=True)

# Quick Verification
print(f"Total Rows: {len(full_df)}")
print(f"Train Rows: {len(train_df)} | Anomalies in Train: {train_df['label'].sum()}")
print(f"Test Rows:  {len(test_df)}  | Anomalies in Test:  {test_df['label'].sum()}")

Total Rows: 28479
Train Rows: 19935 | Anomalies in Train: 2125
Test Rows:  8544  | Anomalies in Test:  569
